<a href="https://colab.research.google.com/github/Karenw1004/NN-Scratch/blob/main/notebooks/Neural_Network_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import required libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
cd drive/MyDrive/NN-Scratch/notebooks/

/content/drive/MyDrive/NN-Scratch/notebooks


# First DataSet: Fashion MNIST
https://www.kaggle.com/zalando-research/fashionmnist 
## Data Preprocessing
Read data

In [3]:
train_df = pd.read_csv("../data/fashion-mnist/fashion-mnist_train.csv")
test_df = pd.read_csv("../data/fashion-mnist/fashion-mnist_test.csv")

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 359.3 MB


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 59.9 MB


In [6]:
train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,0,0,105,92,101,107,100,132,0,0,2,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,...,211,220,214,74,0,255,222,128,0,0,0,0,0,0,0,0,0,44,12,0,0,40,134,162,191,214,163,146,165,79,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,0,114,183,112,55,23,72,102,165,160,28,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,24,188,163,93,...,171,249,207,197,202,45,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,22,21,25,69,52,45,74,39,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,0,0,0,46,0,21,68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,187,189,...,230,237,229,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,116,112,136,147,144,121,102,63,0,0,0,0,0,0,0,0,0,0


There are 1 label col and 784 columns of pixel. <br/>
So, 28(height)*28(width) will results in 784.

In [7]:
train_data = np.array(train_df, dtype='float32')
test_data = np.array(test_df, dtype='float32')

In [8]:
# Ref: https://stackoverflow.com/questions/29831489/convert-array-of-indices-to-1-hot-encoded-numpy-array
def one_hot_encoding_scratch(labels):
  try:
    num_classes = len(np.unique(labels))
    diag = np.eye(num_classes)
    return np.squeeze(diag)[labels.reshape(-1)].T
  except:
    n_values = np.max(labels) + 1
    return np.eye(n_values)[labels]

In [9]:
# train
X_train = train_data[:,1:]
X_train = X_train/255
y_train = train_data[:,0].astype(int)
y_train = one_hot_encoding_scratch(y_train)
# test
X_test = test_data[:, 1:]
X_test = X_test/255
y_test = test_data[:, 0].astype(int)
y_test = one_hot_encoding_scratch(y_test)

Divide by 255 because we need to make data to 0-1. <br />
Do one hot encoding on the labels (y)

#Neural Network
##Layer


In [10]:
class Layer:
  def __init__(input_size, output_size, activation, optimizer):
    self.input = None
    self.output = None
  
  def forward(self, input_data):
    raise ValueError("Not implemented")
  
  def backward(self, deriv_activation, learning_rate, train_size):
    raise ValueError("Not implemented")

Equation for <br/>
Input layer  <br/>
Hidden layer 1   <br/>
Hidden layer 2  <br/>
Output layer   <br/>
With tanh activation  and Softmax

 <br/> <br/>
Feed forward equations: <br/>
z1 = x.w1+b1 <br/>
a1 = tanh(z1) <br/>
z2 = a1.w2+b2 <br/>
a2 = tanh(z2) <br/>
z3 = a2.w3+b3 <br/> 
a3 = softmax(z3) <br/>
<br/><br/>
Back propagation equations: <br/>
There is no z3_delta and softmax_derv(a3), as explained before. <br/>
a3_delta = a3-y     <br/>
z2_delta = a3_delta.w3.T<br/>
a2_delta = z2_delta.tanh_derv(a2) <br/>
z1_delta = a2_delta.w2.T <br/>
a1_delta = z1_delta.tanh_derv(a1) <br/>

With these formulas, let's code

In [11]:
class Dense(Layer):
  
    def __init__(self, input_size, output_size, activation, optimizer):
        self.activation = activation
        self.optimizer = optimizer

        self.weights = np.random.randn(output_size, input_size) * self.activation.heuristic(input_size)
        self.bias = np.ones((output_size, 1))

    def forward(self, input_data):
        self.input = input_data
        self.pre_activation = np.dot(self.weights, self.input) + self.bias
        self.activation_output = self.activation.activ(self.pre_activation)

        return self.activation_output, self.weights

    def backward(self, deriv_activation, learning_rate, train_size):
        deriv_pre_activation = self.activation.deriv(self.activation_output) * deriv_activation
        deriv_activation = np.dot(self.weights.T, deriv_pre_activation)

        self.deriv_weights = np.dot(deriv_pre_activation, self.input.T) / train_size
        self.deriv_bias = np.sum(deriv_pre_activation, axis=1, keepdims=True) / train_size

        self.weights -= learning_rate * (self.optimizer.for_dw(self.deriv_weights))
        self.bias -= learning_rate * self.optimizer.for_db(self.deriv_bias)

        return deriv_activation


In [12]:
class SoftmaxLayer(Dense):
  def backward(self, deriv_activation, learning_rate, train_size):
    deriv_pre_activation = deriv_activation
    deriv_activation = np.dot(self.weights.T, deriv_pre_activation)

    self.deriv_weights = np.dot(deriv_pre_activation, self.input.T) / train_size
    self.deriv_bias = np.sum(deriv_pre_activation, axis=1, keepdims=True) / train_size

    self.weights -= learning_rate * (self.optimizer.for_dw(self.deriv_weights))
    self.bias -= learning_rate * self.optimizer.for_db(self.deriv_bias)

    return deriv_activation


##Loss


In [13]:
class Loss:
  def fct(self, labels, preds):
    raise ValueError("Not implemented")

  def deriv(self, labels, preds):
    raise ValueError("Not implemented")

$mse = \sqrt{(\frac{1}{n})\sum_{i=1}^{n}(y_{i} - x_{i})^{2}}$

In [14]:
class MSE(Loss):
  def fct(self, labels, preds):
    return np.mean(np.power(labels - preds, 2))

  def deriv(self, labels, preds):
    return 2 * (preds - labels) / labels.size

##Optimizer

In [15]:
class Optimizer:
    def __init__(self):
        raise ValueError("Not implemented")

    def for_dw(self, deriv_weights):
        raise ValueError("Not implemented")

    def for_db(self, deriv_bias):
        raise ValueError("Not implemented")

Recommended by the adam paper <br/>
learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08. <br/>

Calculation: <br/>
1. the gradient (partial derivatives) are calculated for the current time step.

> g(t) = f'(x(t-1))

2. the first moment is updated using the gradient and a hyperparameter beta1.

> m(t) = beta1 * m(t-1) + (1 – beta1) * g(t)

3. the second moment is updated using the squared gradient and a hyperparameter beta2.

> v(t) = beta2 * v(t-1) + (1 – beta2) * g(t)^2

The first and second moments are biased because they are initialized with zero values.

In [16]:
class Adam(Optimizer):
    def __init__(self):
        self.var_dw = 0
        self.var_db = 0
        self.mean_dw = 0
        self.mean_db = 0
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.epsilon = 1e-8

    def for_dw(self, deriv_weights):
        self.var_dw = (self.beta1 * self.var_dw) + ((1 - self.beta1) * deriv_weights)
        self.mean_dw = (self.beta2 * self.mean_dw) + ((1 - self.beta2) * np.square(deriv_weights))

        return self.var_dw / (np.sqrt(self.mean_dw) + self.epsilon)

    def for_db(self, deriv_bias):
        self.var_db = (self.beta1 * self.var_db) + ((1 - self.beta1) * deriv_bias)
        self.mean_db = (self.beta2 * self.mean_db) + ((1 - self.beta2) * np.square(deriv_bias))

        return self.var_db / (np.sqrt(self.mean_db) + self.epsilon)

#Activations

In [17]:
class Activation:
  def activ(self, data):
    raise ValueError("Not implemented")

  def deriv(self, data):   
    raise ValueError("Not implemented")

  def heuristic(self, data):
    raise ValueError("Not implemented")


Tanh Function :- The activation that works almost always better than sigmoid function is Tanh function also knows as Tangent Hyperbolic<br/>

f(x) = tanh(x) = 2/(1 + e-2x) - 1
<br/> OR <br />
tanh(x) = 2 * sigmoid(2x) - 1

$\frac{d}{{dx}}\tanh x = 1 - \tanh ^2 x$

In [18]:
class Tanh(Activation):
  def activ(self, data):
    return np.tanh(data)

  def deriv(self, data):
    return 1 - np.square(data)

  def heuristic(self, data):
    return np.sqrt(1 / data)


$
\text{Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}
$

In [19]:
class Softmax(Activation):
  def activ(self, data):
    maxVal = np.max(data, axis=0, keepdims=True) 
    return np.exp(data - maxVal) / np.sum(np.exp(data - maxVal), axis=0, keepdims=True)

  def deriv(self, data):
    return data * (1 - data)

  def heuristic(self, data):
    return np.sqrt(1 / data)


## Network 

In [20]:
class Network:
    def __init__(self):
      self.layers = []
      self.loss = None

    def add(self, layer):
      self.layers.append(layer)

    def use(self, loss):
      self.loss = loss

    def cost(self, loss, size):
      return np.sum(loss) / size

    def train(self, x_train, y_train, epochs, learning_rate, batch_size=128):
      train_size = x_train.shape[1]
      for i in range(1, epochs + 1):

          shuffled_x, shuffled_y = self.shuffle_dataset(x_train, y_train, train_size)

          for batch in range(0, train_size, batch_size):

              batch_a, batch_y = self.get_mini_batch(shuffled_x, shuffled_y, batch, batch_size)

              for layer in self.layers:

                  batch_a, weights = layer.forward(batch_a)

              cost = self.cost(self.loss.fct(batch_y, batch_a), batch_size) 
              deriv_activation = self.loss.deriv(batch_y, batch_a)

              for layer in reversed(self.layers):
                  deriv_activation = layer.backward(deriv_activation,
                                                          learning_rate,
                                                          batch_size)

          print(f"Epoch {i} finished \t {self.loss.__class__.__name__} error {cost}")

    def shuffle_dataset(self, x_train, y_train, train_size):
      permutation = list(np.random.permutation(train_size))
      shuffled_x = x_train[:, permutation]
      shuffled_y = y_train[:, permutation]

      return shuffled_x, shuffled_y

    def get_mini_batch(self, x_train, y_train, pos, batch_size):
      batch_x = x_train[:, pos: pos + batch_size]
      batch_y = y_train[:, pos: pos + batch_size]

      return batch_x, batch_y

    def predict(self, data):
      activation = data
      for layer in self.layers:
          activation, _ = layer.forward(activation)

      return activation
    
    def accuracy(self, x_train, y_train):
      train_results = net.predict(x_train)
      train_results = np.argmax(train_results, axis=0)
      true_results = np.argmax(y_train, axis=0)
      assert len(train_results) == len(true_results)

      return np.mean(train_results == true_results) * 100


# Build network to classify the pictures

In [21]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1]).T
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1]).T

In [22]:
num_classes = y_train.shape[0]
num_classes

10

In [23]:
num_pixels = X_train.shape[0]
num_pixels

784

In [24]:
# Network
net = Network()

net.add(Dense(num_pixels, 100, Tanh(), Adam()))                
net.add(Dense(100, 50, Tanh(), Adam()))                
net.add(Dense(50, 25, Tanh(), Adam()))                
net.add(SoftmaxLayer(25, num_classes, Softmax(), Adam()))                
                  
net.use(MSE())

In [25]:
net.train(X_train, y_train, epochs=10, learning_rate=0.001)

Epoch 1 finished 	 MSE error 0.00011400896386745994
Epoch 2 finished 	 MSE error 0.00016324943188735576
Epoch 3 finished 	 MSE error 0.00011301101678479437
Epoch 4 finished 	 MSE error 0.00013229685611141847
Epoch 5 finished 	 MSE error 0.0001760085791790349
Epoch 6 finished 	 MSE error 0.00012750614131306536
Epoch 7 finished 	 MSE error 7.751138049500242e-05
Epoch 8 finished 	 MSE error 5.9609571658971284e-05
Epoch 9 finished 	 MSE error 5.9893615331036766e-05
Epoch 10 finished 	 MSE error 7.069508842698036e-05


In [26]:
train_acc = net.accuracy(X_train,y_train)
print(f"Train Dataset Accuracy: {train_acc}%")

Train Dataset Accuracy: 91.55166666666666%


In [27]:
test_acc = net.accuracy(X_test,y_test)
print(f"Test Dataset Accuracy: {test_acc}%")

Test Dataset Accuracy: 88.94999999999999%


Good accuracy! Very happy result :)

# Another classification DataSet: Wine DataSet
https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009

In [28]:
df_wine = pd.read_csv("../data/wine/winequality-red.csv")
df_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [29]:
df_wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


**VERY CLEAN DATA**

|Columns| Explanation|
|---|---|
| fixed acidity | most acids involved with wine or fixed or nonvolatile (do not evaporate readily) |
|volatile acidity |the amount of acetic acid in wine, which at too high of levels can lead to an unpleasant, vinegar taste | 
|citric acid|found in small quantities, citric acid can add 'freshness' and flavor to wines|
|residual sugar|the amount of sugar remaining after fermentation stops, it's rare to find wines with less than 1 gram/liter and wines with greater than 45 grams/liter are considered sweet|
|chlorides|the amount of salt in the wine|
|free sulfur dioxide|the free form of SO2 exists in equilibrium between molecular SO2 (as a dissolved gas) and bisulfite ion; it prevents microbial growth and the oxidation of wine|
|total sulfur dioxide|amount of free and bound forms of S02; in low concentrations, SO2 is mostly undetectable in wine, but at free SO2 concentrations over 50 ppm, SO2 becomes evident in the nose and taste of wine|
|density|the density of water is close to that of water depending on the percent alcohol and sugar content|
|pH|describes how acidic or basic a wine is on a scale from 0 (very acidic) to 14 (very basic); most wines are between 3-4 on the pH scale|
|sulphates|a wine additive which can contribute to sulfur dioxide gas (S02) levels, wich acts as an antimicrobial and antioxidant|
|alcohol| alcohol level|
|quality| the quality of the wine|

In [30]:
wine_data = np.array(df_wine, dtype='float32')

In [31]:
y = wine_data[:,11].astype(int)
y

array([5, 5, 5, ..., 6, 5, 6])

In [32]:
y = one_hot_encoding_scratch(y)

In [33]:
y.shape

(1599, 9)

In [34]:
X = wine_data[:,:11]

# Split dataset to 80% train and 20% test

In [35]:
train_size = int(len(X) * 0.8)

In [36]:
X_train = X[:train_size]
X_test = X[train_size:]

y_train = y[:train_size]
y_test = y[train_size:]

In [37]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1]).T
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1]).T
y_train = y_train.reshape(y_train.shape[0],y_train.shape[1]).T
y_test = y_test.reshape(y_test.shape[0],y_test.shape[1]).T

In [38]:
num_classes = y_train.shape[0]
num_rows = X_train.shape[0]

# Build the neural network to classify the quality 

In [39]:
net = Network()

net.add(Dense(num_rows, 64, Tanh(), Adam()))                
net.add(Dense(64, 32, Tanh(), Adam()))                
net.add(Dense(32, 16, Tanh(), Adam()))                
net.add(SoftmaxLayer(16, num_classes, Softmax(), Adam()))                
                  
net.use(MSE())
net.train(X_train, y_train, epochs=10, learning_rate=0.001)

Epoch 1 finished 	 MSE error 0.0005656672606139108
Epoch 2 finished 	 MSE error 0.0005611711086593945
Epoch 3 finished 	 MSE error 0.0005481882484758075
Epoch 4 finished 	 MSE error 0.000540228821009822
Epoch 5 finished 	 MSE error 0.0005077095772468366
Epoch 6 finished 	 MSE error 0.0005485341549348423
Epoch 7 finished 	 MSE error 0.0005352981526697756
Epoch 8 finished 	 MSE error 0.0005295744000108076
Epoch 9 finished 	 MSE error 0.0005351944191345227
Epoch 10 finished 	 MSE error 0.0005084591501289616


In [40]:
train_acc =net.accuracy(X_train,y_train)
print(f"Train Dataset Accuracy: {train_acc}%")

Train Dataset Accuracy: 51.133698201720094%


In [41]:
test_acc = net.accuracy(X_test,y_test)
print(f"Test Dataset Accuracy: {test_acc}%")

Test Dataset Accuracy: 51.5625%


Although it is around ~50%, I am satisfied

Improvements that could be done:
- Save the best model with the lowest loss
- Add more losss
- Add Regularizers
- Add more optimizers 
- Add more activation functions